In [7]:
# Прочитаем датасет и выведем первые пять его строк:

import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

# Давайте посмотрим на информацию о столбцах с помощью метода info():

display(melb_df.info())

# Преобразуем столбец Date в формат datetime и выделим квартал (quarter) продажи объектов недвижимости.
# Найдём второй по популярности квартал продажи

melb_df['Date'] = pd.to_datetime(melb_df['Date'])
quarters = melb_df['Date'].dt.quarter
print(quarters.value_counts().iloc[1])

# Преобразуем все столбцы, в которых меньше 150 уникальных значений, в тип данных category,
# исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
max_unique_count = 150 
for col in melb_df.columns: 
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: 
        melb_df[col] = melb_df[col].astype('category')
display(melb_df.info())

# МЕТОД SORT_VALUES()

# СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА

# Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):

melb_df.sort_values(by='Price').head(10)

# А теперь отсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date). 
# Для этого выставим параметр ascending на False:

melb_df.sort_values(by='Date', ascending=False)

# СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

# Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр by в виде списка.
# При этом важно обращать внимание на порядок следования столбцов.

# отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance),
# а затем — по возрастанию цены объекта (Price). Для того чтобы вывод был более наглядным,
# выделим каждую десятую строку из столбцов Distance и Price результирующей таблицы:

melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

# Мы получили таблицу, отсортированную по возрастанию расстояния до центра города. 
# Если встречаются объекты недвижимости, у которых расстояние оказывается одинаковым, 
# то внутри такой группы производится сортировка по цене объекта.

# КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ

# Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов,
# у которых площадь участка существенно больше площади здания, чтобы понять,
# как часто компания справляется с таким сложным видом объектов. Объекты, проданные в один и тот же день,
# мы бы хотели сортировать по значению коэффициента соотношения площадей.

# Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath,
# у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8.
# Результат отсортируем по дате продажи (Date) в порядке возрастания,
# а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые,
# установив параметр ignore_index на True. Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio: 

mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

# Отсортируем таблицу по столбцу AreaRatio с параметром ascending=False (по убыванию).
# Выставим параметр ignore_index на True, чтобы заменить индексы на новые.
# С помощью метода loc извлечем значение столбца BuildingArea в строке с индексом 1558.

int(melb_df.sort_values(
    by='AreaRatio', 
    ignore_index=True,
    ascending=False
).loc[1558, 'BuildingArea'])

# Произведем фильтрацию таблицы и отберите строки по условиям «тип здания — таунхаус» и «число жилых комнат больше 2».
# Отсортируем полученную таблицу по столбцу Rooms с параметром ascending=True (по возрастанию)
# и по средней площади комнат с параметром ascending=False (по убыванию).
# Выставим параметр ignore_index на True, чтобы заменить индексы на новые.
# С помощью метода loc извлечем значение столбца Price в строке с индексом 18.

mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
int(melb_df[mask1&mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price'])

# Группировка данных в DataFrame

# Одна из основных задач анализа данных — это группировка данных и сравнение показателей в группах.
# Например, нам необходимо сравнить средний уровень заработной платы в зависимости от пола/уровня образования.
# Или же мы хотим проследить, какая группа клиентов приносит нам наибольший доход, чтобы направить своё внимание на эту группу.

# МЕТОД GROUPBY()

# by — имя или список имён столбцов, по которым производится группировка.
# axis — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
# as_index — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.

#ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

# Применим агрегирующую функцию среднего к результату работы groupby().
# В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):

melb_df.groupby(by='Type').mean()

# Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только к интересующему нас столбцу.
# Например, давайте сравним средние цены на объекты в зависимости от их типа:

melb_df.groupby('Type')['Price'].mean()

# Из этой маленькой таблицы видно, что наибольшей средней ценой обладают объекты типа house (дома, коттеджи, виллы).
# Следовательно, можно сделать вывод, что тип постройки является значимым фактором при определении цены объекта недвижимости.

# Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.

# Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона.
# Результат отсортируем по убыванию расстояния:

melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

# Итак, наиболее удалёнными являются все регионы Victoria.

# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

# Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(),
# который принимает список строк с названиями агрегаций.

# Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж,
# а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale).
# Результат отсортируем по количеству продаж в порядке убывания:

melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

# так как мы считаем несколько показателей для одного столбца, в результате мы получаем объект DataFrame.

# В результате применения метода agg(), в который мы передали список с названиями интересующих нас агрегирующих функций,
# мы получаем DataFrame со столбцами count, mean и max, где для каждого месяца рассчитаны соответствующие параметры.
# Результат сортируем по столбцу count.

# Какие интересные выводы можно сделать из этой таблицы?

# 1 - Пик продаж приходится на период весна-лето.
# 2 - Средняя цена продаваемых объектов относительно стабильна и находится в пределах 1 млн. австралийских долларов
#     с небольшими отклонениями (около 100 тыс. влево и вправо).
# 3 - Прослеживается некоторая зависимость между сезоном и максимальной ценой объектов: в месяцы с большим спросом
#     на объекты недвижимости цена также имеет наибольшие показатели. Можно сделать предположение, что это связано с повышением
#     цен на элитные дома в периоды большого спроса.

# Если нам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы,
# Мы можем воспользоваться методом agg(), передав в качестве его параметра строку 'describe':

melb_df.groupby('MonthSale')['Price'].agg('describe')

# После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений.
# Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять,
# в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 

# Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set,
# чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set])

# Как и ожидалось, наименьшая конкуренция в наиболее удалённом регионе Western Victoria,
# а наибольшая — в центральном районе Northern Metropolitan.





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

None

4359
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  

None

C:\Users\Konstantin\AppData\Local\Temp\ipykernel_33796\2411172429.py:120: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  melb_df.groupby(by='Type').mean()


,nunique,set
Regionname,,
Eastern Metropolitan,26,"{Gary, McGrath, Biggin, Purplebricks, Harcourt..."
Eastern Victoria,11,"{other, McGrath, Harcourts, O'Brien, HAR, hock..."
Northern Metropolitan,40,"{Biggin, McGrath, Purplebricks, Harcourts, Vil..."
Northern Victoria,11,"{other, McGrath, LITTLE, HAR, YPA, hockingstua..."
South-Eastern Metropolitan,25,"{Gary, McGrath, Biggin, Purplebricks, Harcourt..."
Southern Metropolitan,38,"{Gary, Biggin, McGrath, Purplebricks, Harcourt..."
Western Metropolitan,34,"{Biggin, McGrath, Purplebricks, Harcourts, Bel..."
Western Victoria,6,"{other, YPA, HAR, hockingstuart, Ray, Raine}"
